# Calibration Diagnostics and Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-username/aquacal/blob/main/docs/tutorials/02_diagnostics.ipynb)

This tutorial shows you how to interpret calibration quality and diagnose issues using reprojection error analysis, parameter convergence plots, and 3D rig visualization.

**What you'll learn:**
- Understanding reprojection error metrics and what good values look like
- Spatial error analysis across the image plane
- Parameter convergence and interface distance estimation
- 3D visualization of camera rig geometry
- Common failure modes and how to fix them

In [ ]:
# Data source toggle: choose which dataset to use
DATASET = "small"  # Options: "small" (fast, included), "medium", "large"

# For Google Colab: uncomment to install aquacal
# !pip install aquacal

## Setup and Data Loading

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from aquacal.config.schema import InterfaceParams
from aquacal.core.board import BoardGeometry
from aquacal.datasets import generate_synthetic_rig
from tests.synthetic.ground_truth import generate_synthetic_detections

print(f"Loading {DATASET} dataset...")

# Generate synthetic scenario with known ground truth
scenario = generate_synthetic_rig(DATASET)
print(f"  {len(scenario.intrinsics)} cameras")
print(f"  {len(scenario.board_poses)} frames")
print(f"  {scenario.noise_std}px detection noise")

# Create board geometry
board = BoardGeometry(scenario.board_config)

# Generate synthetic detections
detections = generate_synthetic_detections(
    intrinsics=scenario.intrinsics,
    extrinsics=scenario.extrinsics,
    water_zs=scenario.water_zs,
    board=board,
    board_poses=scenario.board_poses,
    noise_std=scenario.noise_std,
    seed=42,
)

print("\nRunning calibration pipeline...")

In [ ]:
# Run calibration (Stages 2-4)
from aquacal.calibration.extrinsics import build_pose_graph, estimate_extrinsics
from aquacal.calibration.interface_estimation import optimize_interface
from aquacal.calibration.refinement import joint_refinement
from aquacal.config.schema import (
    CalibrationMetadata,
    CalibrationResult,
    CameraCalibration,
    DiagnosticsData,
)
from aquacal.validation.reprojection import compute_reprojection_errors

reference_camera = "cam0"
interface_normal = np.array([0.0, 0.0, -1.0], dtype=np.float64)

# Stage 2: Extrinsic initialization
print("  Stage 2: Extrinsic initialization...")
pose_graph = build_pose_graph(detections, min_cameras=2)
initial_extrinsics = estimate_extrinsics(
    pose_graph, scenario.intrinsics, board, reference_camera
)

# Stage 3: Joint refractive optimization
print("  Stage 3: Joint refractive optimization...")
opt_extrinsics, opt_distances, opt_poses, rms = optimize_interface(
    detections=detections,
    intrinsics=scenario.intrinsics,
    initial_extrinsics=initial_extrinsics,
    board=board,
    reference_camera=reference_camera,
    interface_normal=interface_normal,
    n_air=1.0,
    n_water=1.333,
    loss="huber",
    loss_scale=1.0,
    min_corners=4,
)

# Stage 4: Intrinsic refinement
print("  Stage 4: Intrinsic refinement...")
stage3_result = (opt_extrinsics, opt_distances, opt_poses, rms)
opt_extrinsics, opt_distances, opt_poses, opt_intrinsics, rms = joint_refinement(
    stage3_result=stage3_result,
    detections=detections,
    intrinsics=scenario.intrinsics,
    board=board,
    reference_camera=reference_camera,
    refine_intrinsics=True,
    interface_normal=interface_normal,
    n_air=1.0,
    n_water=1.333,
    loss="huber",
    loss_scale=1.0,
    min_corners=4,
)

# Build CalibrationResult
cameras = {}
for cam_name in scenario.intrinsics:
    cameras[cam_name] = CameraCalibration(
        name=cam_name,
        intrinsics=opt_intrinsics[cam_name],
        extrinsics=opt_extrinsics[cam_name],
        water_z=opt_distances[cam_name],
    )

interface_params = InterfaceParams(
    normal=interface_normal,
    n_air=1.0,
    n_water=1.333,
)

# Compute per-camera RMS errors

per_camera_rms = {}
for cam_name in cameras:
    cam_errors = compute_reprojection_errors(
        calibration=cameras[cam_name],
        interface_params=interface_params,
        detections=detections,
        board=board,
    )
    per_camera_rms[cam_name] = np.sqrt(np.mean(cam_errors**2))

diagnostics = DiagnosticsData(
    reprojection_error_rms=rms,
    reprojection_error_per_camera=per_camera_rms,
    validation_3d_error_mean=0.0,
    validation_3d_error_std=0.0,
)

metadata = CalibrationMetadata(
    calibration_date="synthetic",
    software_version="test",
    config_hash="synthetic",
    num_frames_used=len(opt_poses),
    num_frames_holdout=0,
)

result = CalibrationResult(
    cameras=cameras,
    interface=interface_params,
    board=scenario.board_config,
    diagnostics=diagnostics,
    metadata=metadata,
)

print("\nCalibration complete!")
print(f"  Overall RMS: {result.diagnostics.reprojection_error_rms:.3f} px")

## Reprojection Error Analysis

Reprojection error measures how well the calibrated model predicts the observed corner positions. Low values (<1 px for good detections) indicate accurate calibration.

**What good values look like:**
- Overall RMS: < 1.0 px (excellent), < 2.0 px (good)
- Per-camera variation: cameras should have similar RMS values
- Spatial uniformity: errors should be distributed evenly across the image plane

In [ ]:
# Per-camera RMS error bar chart
camera_names = sorted(result.cameras.keys())
rms_values = [
    result.diagnostics.reprojection_error_per_camera[cam] for cam in camera_names
]

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(camera_names, rms_values, color="steelblue", alpha=0.8)
ax.axhline(
    result.diagnostics.reprojection_error_rms,
    color="red",
    linestyle="--",
    label="Overall RMS",
)
ax.set_xlabel("Camera")
ax.set_ylabel("RMS Reprojection Error (px)")
ax.set_title("Per-Camera Reprojection Error")
ax.legend()
ax.grid(axis="y", alpha=0.3)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
plt.close()

print(
    "\n⚠️  **Warning:** High error in one camera often indicates poor intrinsics or insufficient board observations for that camera."
)

In [ ]:
# Collect all reprojection errors for distribution analysis
all_errors = []
spatial_errors = []  # For spatial heatmap: (u, v, error)

for cam_name in camera_names:
    cam = result.cameras[cam_name]
    errors = compute_reprojection_errors(
        calibration=cam,
        interface_params=result.interface,
        detections=detections,
        board=board,
    )

    # Compute error magnitudes
    error_mags = np.linalg.norm(errors, axis=1)
    all_errors.extend(error_mags)

    # Store spatial data (u, v, error) for first camera only (for heatmap demo)
    if cam_name == camera_names[0]:
        for frame_idx, frame_det in detections.frames.items():
            if cam_name in frame_det.detections:
                obs = frame_det.detections[cam_name]
                for i in range(len(obs.corner_ids)):
                    u, v = obs.image_points[i]
                    spatial_errors.append(
                        (u, v, error_mags[i] if i < len(error_mags) else 0)
                    )

# Histogram of error distribution
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(all_errors, bins=30, color="steelblue", alpha=0.7, edgecolor="black")
ax.axvline(
    np.mean(all_errors),
    color="red",
    linestyle="--",
    label=f"Mean: {np.mean(all_errors):.2f} px",
)
ax.set_xlabel("Reprojection Error (px)")
ax.set_ylabel("Frequency")
ax.set_title("Reprojection Error Distribution (All Cameras)")
ax.legend()
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

print("Error statistics:")
print(f"  Mean: {np.mean(all_errors):.3f} px")
print(f"  Median: {np.median(all_errors):.3f} px")
print(f"  95th percentile: {np.percentile(all_errors, 95):.3f} px")
print(f"  Max: {np.max(all_errors):.3f} px")

In [ ]:
# Spatial error heatmap (for first camera as example)
if len(spatial_errors) > 0:
    spatial_errors = np.array(spatial_errors)
    u_coords = spatial_errors[:, 0]
    v_coords = spatial_errors[:, 1]
    errors = spatial_errors[:, 2]

    fig, ax = plt.subplots(figsize=(10, 8))
    scatter = ax.scatter(
        u_coords,
        v_coords,
        c=errors,
        cmap="hot",
        s=50,
        alpha=0.6,
        edgecolors="black",
        linewidth=0.5,
    )
    ax.set_xlabel("u (pixels)")
    ax.set_ylabel("v (pixels)")
    ax.set_title(f"Spatial Error Distribution ({camera_names[0]})")
    ax.invert_yaxis()  # Image coordinate convention
    ax.set_aspect("equal")
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label("Reprojection Error (px)")
    plt.tight_layout()
    plt.show()
    plt.close()

    print("\nSpatial error heatmap shows error distribution across the image plane.")
    print(
        "Clusters of high error may indicate distortion modeling issues or board pose degeneracies."
    )

## Parameter Convergence

For synthetic data, we can compare estimated parameters to ground truth. This shows how well the optimization converged to the true values.

**Interface distance** is the Z-coordinate of the water surface in world frame (meters). All cameras should estimate similar values after optimization.

In [ ]:
# Compare estimated interface distances to ground truth
estimated_distances = [result.cameras[cam].water_z for cam in camera_names]
gt_distances = [scenario.water_zs[cam] for cam in camera_names]

fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(camera_names))
width = 0.35

ax.bar(
    x - width / 2,
    estimated_distances,
    width,
    label="Estimated",
    color="steelblue",
    alpha=0.8,
)
ax.bar(
    x + width / 2, gt_distances, width, label="Ground Truth", color="orange", alpha=0.8
)

ax.set_xlabel("Camera")
ax.set_ylabel("Interface Distance (m)")
ax.set_title("Interface Distance Recovery")
ax.set_xticks(x)
ax.set_xticklabels(camera_names, rotation=45, ha="right")
ax.legend()
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

mean_error = np.mean(
    [abs(est - gt) for est, gt in zip(estimated_distances, gt_distances)]
)
print(f"\nMean absolute interface distance error: {mean_error * 1000:.2f} mm")
print(
    "\n⚠️  **Warning:** If interface distance diverges or oscillates, the initial estimate may be too far from truth. Try providing explicit initial_distances in config."
)

In [ ]:
# Compare camera positions (Z-coordinate) to ground truth
estimated_z = [result.cameras[cam].extrinsics.C[2] for cam in camera_names]
gt_z = [scenario.extrinsics[cam].C[2] for cam in camera_names]

fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(camera_names))
width = 0.35

ax.bar(
    x - width / 2, estimated_z, width, label="Estimated", color="steelblue", alpha=0.8
)
ax.bar(x + width / 2, gt_z, width, label="Ground Truth", color="orange", alpha=0.8)

ax.set_xlabel("Camera")
ax.set_ylabel("Camera Z Position (m)")
ax.set_title("Camera Z Position Recovery")
ax.set_xticks(x)
ax.set_xticklabels(camera_names, rotation=45, ha="right")
ax.legend()
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

z_errors = [abs(est - gt) * 1000 for est, gt in zip(estimated_z, gt_z)]
print(f"\nCamera Z position errors (mm): {[f'{e:.2f}' for e in z_errors]}")
print(f"Mean: {np.mean(z_errors):.2f} mm, Max: {np.max(z_errors):.2f} mm")

## 3D Rig Visualization

Visualizing the camera rig in 3D helps verify that the estimated geometry is physically plausible.

In [ ]:
# 3D plot of camera rig
fig = plt.figure(figsize=(12, 10))

# Top view
ax1 = fig.add_subplot(2, 2, 1)
for cam_name in camera_names:
    C_est = result.cameras[cam_name].extrinsics.C
    C_gt = scenario.extrinsics[cam_name].C
    ax1.scatter(
        C_est[0], C_est[1], marker="o", s=100, label=f"{cam_name} (est)", alpha=0.7
    )
    ax1.scatter(
        C_gt[0], C_gt[1], marker="x", s=100, label=f"{cam_name} (gt)", alpha=0.7
    )
    ax1.plot([C_est[0], C_gt[0]], [C_est[1], C_gt[1]], "k--", alpha=0.3)

ax1.set_xlabel("X (m)")
ax1.set_ylabel("Y (m)")
ax1.set_title("Top View (XY Plane)")
ax1.set_aspect("equal")
ax1.grid(alpha=0.3)

# Side view (XZ)
ax2 = fig.add_subplot(2, 2, 2)
for cam_name in camera_names:
    C_est = result.cameras[cam_name].extrinsics.C
    C_gt = scenario.extrinsics[cam_name].C
    ax2.scatter(C_est[0], C_est[2], marker="o", s=100, alpha=0.7)
    ax2.scatter(C_gt[0], C_gt[2], marker="x", s=100, alpha=0.7)
    ax2.plot([C_est[0], C_gt[0]], [C_est[2], C_gt[2]], "k--", alpha=0.3)

# Water surface (average interface distance)
water_z = np.mean(estimated_distances)
ax2.axhline(water_z, color="blue", linestyle="--", alpha=0.5, label="Water surface")

ax2.set_xlabel("X (m)")
ax2.set_ylabel("Z (m)")
ax2.set_title("Side View (XZ Plane)")
ax2.set_aspect("equal")
ax2.grid(alpha=0.3)
ax2.legend()

# 3D view
ax3 = fig.add_subplot(2, 2, 3, projection="3d")
for cam_name in camera_names:
    C_est = result.cameras[cam_name].extrinsics.C
    C_gt = scenario.extrinsics[cam_name].C
    ax3.scatter(
        C_est[0],
        C_est[1],
        C_est[2],
        marker="o",
        s=100,
        label=f"{cam_name} (est)",
        alpha=0.7,
    )
    ax3.scatter(C_gt[0], C_gt[1], C_gt[2], marker="x", s=100, alpha=0.7)

# Water surface plane
xlim = ax3.get_xlim()
ylim = ax3.get_ylim()
xx, yy = np.meshgrid(xlim, ylim)
zz = np.full_like(xx, water_z)
ax3.plot_surface(xx, yy, zz, color="blue", alpha=0.2)

ax3.set_xlabel("X (m)")
ax3.set_ylabel("Y (m)")
ax3.set_zlabel("Z (m)")
ax3.set_title("3D View")

plt.tight_layout()
plt.show()
plt.close()

print("\nBlue circles: estimated camera positions")
print("Orange crosses: ground truth camera positions")
print("Dashed lines: position errors")

## Validation Metrics

3D reconstruction error: triangulate known board corners and compare to true spacing.

In [ ]:
# Evaluate 3D reconstruction accuracy
from aquacal.validation.reconstruction import compute_3d_distance_errors

dist_errors = compute_3d_distance_errors(
    calibration=result,
    detections=detections,
    board=board,
    include_per_pair=False,
    include_spatial=True,
)

print("\n3D Reconstruction Quality:")
print(f"  Signed mean error: {dist_errors.signed_mean * 1000:.3f} mm")
print(f"  RMSE: {dist_errors.rmse * 1000:.3f} mm")
print(
    f"  Scale factor (measured/true): {1.0 + dist_errors.signed_mean / board.config.square_size:.6f}"
)
print(f"  Number of comparisons: {dist_errors.num_comparisons}")

# Histogram of distance errors
if dist_errors.spatial is not None:
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.hist(
        dist_errors.spatial.signed_errors * 1000,
        bins=30,
        color="steelblue",
        alpha=0.7,
        edgecolor="black",
    )
    ax.axvline(0, color="black", linestyle="--", alpha=0.5)
    ax.axvline(
        dist_errors.signed_mean * 1000,
        color="red",
        linestyle="--",
        label=f"Mean: {dist_errors.signed_mean * 1000:.2f} mm",
    )
    ax.set_xlabel("Signed Distance Error (mm)")
    ax.set_ylabel("Frequency")
    ax.set_title("3D Reconstruction Error Distribution")
    ax.legend()
    ax.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()
    plt.close()

## Common Issues Checklist

| Symptom | Likely Cause | Fix |
|---------|-------------|-----|
| High error for one camera | Poor intrinsic calibration quality | Re-calibrate intrinsics with more frames, check for motion blur |
| High error in image corners | Distortion model insufficient | Try rational model (8 coefficients) or fisheye model |
| Interface distance not converging | Initial estimate too far from truth | Provide better `initial_water_zs` in config |
| Interface distances differ between cameras | Degenerate board poses | Ensure board is visible at varied angles and depths |
| Spatial error clusters | Board pose degeneracies | Add more frames with board at different orientations |
| High 3D reconstruction error | Systematic bias in parameters | Check that n_water is correct for your water type |
| Reprojection < 1px but 3D error high | Overfitting to 2D, wrong geometry | Verify interface distance initialization, add validation frames |

---

**Next steps:**
- If errors are high: review intrinsic calibration, add more extrinsic frames
- If parameters don't converge: improve initialization estimates
- If everything looks good: proceed to full dataset calibration!